In [ ]:
from SmoothPursuitClassification import SmoothPursuitClassification

import re, warnings
warnings.filterwarnings('ignore')

import numpy
import scipy, scipy.stats
import pandas
from pandas import DataFrame

%matplotlib inline
#%matplotlib tk
#%gui

import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [ ]:
spc = SmoothPursuitClassification()
spc.settingsReader.select('../dataset/zhegallo/01-1Settings.xml')
spc.dataReader.read(spc.settingsReader, spc.multiData)

samplesData = spc.multiData.getChannelAndTag('samples', '01-1', block='trial', ignoreEmpty=False)
velocityData = spc.multiData.getVelocity(samplesData=samplesData, smooth='savgol', convertToDeg=True)

In [ ]:
samplesData.columns
#velocityData.loc[:, ('Time', 'L POR X [px]', 'L POR Y [px]', 'R POR X [px]', 'R POR Y [px]', 'LVelocity', 'RVelocity')]

#spc.dataExporter.exportCSV(spc.multiData)

In [ ]:
#style settings
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
matplotlib.style.use('bmh') #bmh, ggplot, seaborn, fivethirtyeight
params = {#'font.family': 'arial',
          'figure.titleweight': 'bold',
          'figure.figsize': (6, 6),
          'figure.dpi': 300,
          #'savefig.dpi': 300,
          'axes.titlesize': 'medium',
          'axes.titleweight': 'bold',
          'axes.labelsize': 'small',
          #'axes.labelweight': 'normal',
          'axes.facecolor': '#E8DDCB',
          'axes.grid': True,
          'axes.grid.which': 'both',    #both, major
          'axes.axisbelow': True,    #line, False
          'grid.linewidth': 0.5,
          'grid.alpha': 0.8,
          'xtick.labelsize': 'x-small',
          'ytick.labelsize': 'x-small',
          'xtick.direction': 'out',
          'ytick.direction': 'out',
          'ytick.left': True,
          'xtick.bottom': True,
          'ytick.minor.left': True,
          'xtick.minor.bottom': True,
          'xtick.minor.visible': True,
          'ytick.minor.visible': True,
          'legend.fontsize': 'xx-small',
          'legend.facecolor': 'white'
          }
pylab.rcParams.update(params)

import seaborn as sns
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               #'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               #'font.family': ['sans-serif'],
               'grid.color': '.8',
               #'grid.linestyle': '-',
               'image.cmap': 'rocket',
               #'lines.solid_capstyle': 'round',
               #'patch.edgecolor': 'w',
               #'patch.force_edgecolor': True,
               #'text.color': '.15',
               'xtick.bottom': True,
               #'xtick.color': '.15',
               'xtick.direction': 'out',
               'xtick.top': False,
               #'ytick.color': '.15',
               'ytick.direction': 'out',
               'ytick.left': True,
               'ytick.right': False})

In [ ]:
#preparing data
plotData = velocityData.loc[(velocityData['Time']>=12.000) & (velocityData['Time']<13.000), :]
#res = (velocityData['Time']>=6.000) & (velocityData['Time']<6.800)
#plotData['L POR X [px]']

In [ ]:
#PLOTTING methods
#seaborn lineplot default sort=True
plt.figure(figsize=(10, 15))
plt.suptitle('COMBINED PLOT')
plt.subplot(311)
plt.title('X,Y-t')
plt.axis(ymin=-20, ymax=20)
sns.lineplot(plotData['Time'], plotData['LPORXDeg'], label='Left X raw', color='pink', linewidth=1, sort=False)
sns.lineplot(plotData['Time'], plotData['LPORYDeg'], label='Left Y raw', color='skyblue', linewidth=1, sort=False)
#sns.lineplot(plotData['Time'], plotData['LPORXDegSmoothed'], label='Left X smoothed', color='red', linewidth=1)
#sns.lineplot(plotData['Time'], plotData['LPORYDegSmoothed'], label='Left Y smoothed', color='blue', linewidth=1)
plt.xlabel('Time (s)')
plt.ylabel('Gaze position (degrees)')

plt.subplot(312)
plt.title('Velocity')
plt.axis(ymin=0, ymax=1000)
sns.lineplot(plotData['Time'], plotData['LVelocity'], label='Left velocity', color='orange', linewidth=1, sort=False)
plt.xlabel('Time (s)')
plt.ylabel('Velocity (degrees/s)')

plt.subplot(313)
plt.title('Position')
plt.axis(xmin=-20, xmax=20, ymin=-20, ymax=20)
sns.lineplot(plotData['LPORXDeg'], plotData['LPORYDeg'], label='Left position raw', sort=False)
#sns.lineplot(plotData['LPORXDegSmoothed'], plotData['LPORYDegSmoothed'], label='Left position smoothed', color='black', linewidth=0.5)
plt.xlabel('X (degrees)')
plt.ylabel('Y (degrees)')

In [ ]:
#PLOTLY
trace1 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['L POR X [px]'],
                    mode = 'lines',
                    name = 'X-t pixels'
                   )
trace2 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['L POR Y [px]'],
                    mode = 'lines',
                    name = 'Y-t pixels'
                   )
layout = go.Layout(scene=dict(dict(xaxis=dict(title='time'), 
                                   yaxis=dict(title='X,Y')
                                  )
                             )
                  )
figData = [trace1, trace2]
fig1 = dict(data=figData, layout=layout)
py.iplot(fig1)

In [ ]:
trace11 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['LPORXDeg'],
                    mode = 'lines',
                    name = 'X-t raw'
                   )
trace12 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['LPORYDeg'],
                    mode = 'lines',
                    name = 'Y-t raw'
                   )
trace21 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['LPORXDegSmoothed'],
                    mode = 'lines',
                    name = 'X-t smoothed'
                   )
trace22 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['LPORYDegSmoothed'],
                    mode = 'lines',
                    name = 'Y-t smoothed'
                   )
layout = go.Layout(scene=dict(dict(xaxis=dict(title='time'), 
                                   yaxis=dict(title='X,Y')
                                  )
                             )
                  )
figData = [trace11, trace12]
fig1 = dict(data=figData, layout=layout)
py.iplot(fig1)

In [ ]:
trace1 = go.Scatter(x = velocityData['Time'],
                    y = velocityData['LVelocity'],
                    mode = 'lines',
                    name = 'Left velocity'
                   )
layout = go.Layout(scene=dict(dict(xaxis=dict(title='time'), 
                                   yaxis=dict(title='Velocity')
                                  )
                             )
                  )
figData = [trace1]
fig1 = dict(data=figData, layout=layout)
py.iplot(fig1)

In [ ]:
#3D
trace1 = go.Scatter3d(x=plotData['LPORXDegSmoothed'],
                      y=plotData['LPORYDegSmoothed'],
                      z=plotData['LVelocity'],
                      mode='lines'
                     )
layout = go.Layout(scene=dict(dict(xaxis=dict(title='X'), 
                                   yaxis=dict(title='Y'),
                                   zaxis=dict(title='Velocity')
                                  )
                             )
                  )
figData = [trace1]
fig1 = dict(data=figData, layout=layout)
#py.plot(fig1, filename='scatter3d01.html', auto_open=True)
py.iplot(fig1)

In [ ]:
#PAIRPLOT
pairData = plotData.filter(('LPORXDegSmoothed','LPORYDegSmoothed','LVelocity', 'Trial'))
ax = sns.pairplot(pairData, diag_kind='hist', hue='Trial')